<a href="https://colab.research.google.com/github/MarcoFidelVasquezRivera/K-nearest-Neighbour/blob/Marco/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import pandas as pd
import sklearn #se importa para obtener un dataset
import matplotlib.pyplot as plt#se importa para poder realizar la gráfica con los datos

from sklearn import datasets
#obtiene los datos desde iris
iris = datasets.load_iris()
iris

{'DESCR': '.. _iris_dataset:\n\nIris plants dataset\n--------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n                \n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)\n    ===========

In [ ]:
X = iris.data[:, :2]
Y = iris.target
plt.scatter(X[:,0],X[:,1])

In [36]:
import math
#definimos la clase KNN
class KNN:

  def __init__(self,k):
    self.k = k

  def euclidean_distance(self, idx1, idx2, data, axis=0):
    """    Returns the euclidean distance between x and y.

            Preconditions:
                    the dimensions of x and y must be the same

            Parameters:
                    x (n-dimensional numpy array): an array of integers
                    y (n-dimensional numpy array): an array of integers
                    axis (int): axis in which we are working

            Returns:
                    int): distance between x and y on the axis axis.
    """
    return np.linalg.norm( data[idx1] - data[idx2], axis=axis)

  def get_k_nearest_neighboors(self, idx, data):
        """
        Returns the K nearest neighboors of the point.

        Preconditions:
                the dimensions of point and the points on data must be the same

        Parameters:
                idx (int): the index of the point which we are going to get the distance with the other points
                data: an n-dimensional numpy array
        Postcondition:
                the distance of this point to itself is not going to be returned
        Returns:
                idxs (int): an array whit the indexes of the k nearest neighboors of the point with index idx.
        """
        # Calculate the distances from datapoint in idx with every other datapoint in data 
        distances = np.array([  self.euclidean_distance(idx, i, data) for i in range(len(data)) ])
        # Maps from distance to the index in data excluding idx
        index_distance = dict((distance, index) for index, distance in enumerate(distances) if index != idx)
        distances = np.delete(distances, idx)
        distances.sort()
        top_k_nearest_neighbors = np.array([ index_distance[distance] for distance in distances])
        response = []

        i = 0
        # This ensures that only non-duplicate indexes are returned
        while len(response) < self.k:
          kth_nearest = top_k_nearest_neighbors[i]
          if kth_nearest not in response:
            response.append(kth_nearest)
          i+=1
        return np.array(response)

  def set_point_target(self,indexes,target_data):
    #falta hacer la documentación
    frequency = dict((k,0) for k in target_data)

    for index in indexes:
      result = target_data[index]
      frequency[result]+=1
    
    most_freq = -1
    classfreq = -1
    for clas,freq in frequency.items():
      if freq > most_freq:
        most_freq = freq
        classfreq = clas

    return classfreq

    # fit
  def fit(self, X, targets):
    # k 
    self.data = X
    self.index_to_class = {} # inicializa un arreglo que mapea, desde los indixes
    for datapoint_idx in range(len(X)):
      k_nearest = self.get_k_nearest_neighboors(datapoint_idx,self.data)
      classfreq = self.set_point_target(k_nearest,targets)
      self.index_to_class[datapoint_idx] = classfreq

  def predict(self,data):
    if data.shape[-1]==self.index_to_class.shape[-1]:

      for datapoint_idx in len(data):
        matrix = np.concatenate(self.data,data[datapoint_idx])
        k_nearest = self.get_k_nearest_neighboors(matrix[-1],matrix)
        





In [35]:
X.shape


(150, 2)

In [ ]:
#lo probamos haciendo uso de la base de datos cargada anteriormente
knn = KNN(10)
knn.fit(X,Y)
knn.index_to_class

In [41]:
matrix = [[1.8,2.4],[2.3,1.4]]
array = [3.4,5.6]
#new_matrix = np.concatenate(matrix,array[0])
#new_matrix